# Assignment 2: Privacy-Preserving Data Sharing
### Security and Privacy Project - MECD (2022/2023)

### Autores

- Duarte Meneses - 2019216949
- Patricia Costa - 2019213995

### Import dataset

In [117]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

infri = pd.read_csv('data/infringement_dataset_v2.csv')

infri.fillna(value = -1,  
          inplace = True) 

display(infri)

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,first_name,last_name,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,Robert,Watkins,9251.77500,179055.000,179055.00,1.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,Jane,Navarro,56553.99000,435436.500,484191.00,3.0,0.0,0.0,0.0,3.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,David,Seagraves,5357.25000,24282.000,20106.00,1.0,0.0,0.0,0.0,1.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,Deborah,Tandy,23651.17500,272203.260,291695.50,5.0,3.0,1.0,0.0,9.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,David,Walker,12278.80500,150530.250,166638.75,6.0,0.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,Lawrence,Mcwayne,6605.91000,40455.000,40455.00,1.0,0.0,0.0,0.0,1.0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,Belinda,Loveland,10074.46500,57595.500,56821.50,1.0,0.0,0.0,0.0,1.0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,Aileen,Rasmussen,4770.40500,24162.750,20625.75,2.0,0.0,0.0,0.0,2.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,Mary,Marbury,10681.13250,121317.750,134439.75,2.0,0.0,0.0,0.0,2.0


### 2.2. Sensitivity

##### Mean of past events

In [124]:
from matplotlib import pyplot as plt
columns = ['past_avg_amount_annuity', 'past_avg_amt_application', 'past_avg_amt_credit', 'past_loans_total']

In [128]:
def sensitivity_mean(dataset):

    l = []
    mean_past = dataset.mean()
    print("Total mean:", mean_past)

    for i in range(len(dataset)):
        if i % 1000 == 0:
            print(i)
        aux = dataset.drop(dataset.index[i])
        l.append(abs(mean_past - aux.mean()))

    return max(l)

df_sensitivity = pd.DataFrame(columns=columns, index = ["Sensitivity"])

s_annuity = sensitivity_mean(infri['past_avg_amount_annuity'])
s_application = sensitivity_mean(infri['past_avg_amt_application'])
s_credit = sensitivity_mean(infri['past_avg_amt_credit'])
s_loans = sensitivity_mean(infri['past_loans_total'])

df_sensitivity["Sensitivity"] = [s_annuity, s_application, s_credit, s_loans]

display(df_sensitivity)

Total mean: 13734.724771635369
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000


KeyboardInterrupt: 

##### Mean of people age of infringements

In [120]:
df = infri[['infringed', 'age']]
display(df)

,infringed,age
0,1,25
1,0,45
2,0,52
3,0,52
4,0,54
...,...,...
307506,0,25
307507,0,56
307508,0,41
307509,1,32


In [121]:
analyse = df[df['infringed'] == 1]
original_count = len(analyse)

l = []
mean_age = analyse['age'].mean()
print("Total mean:", mean_age)
for i in range(original_count):
    aux = analyse.drop(analyse.index[i])
    l.append(abs(mean_age - aux['age'].mean()))

sensitivity_age = max(l)
print("Sensitivity: ", sensitivity_age)


Total mean: 40.279959718026184
Sensitivity:  0.0011166629182213228


### 2.3. Differential Privacy

In [122]:
def add_laplace_noise(data, sensivity_v, epsilon):
  noisy_data = data + np.random.laplace(scale=(sensivity_v / epsilon))
  return noisy_data

def percentage_error(orig, est):
  return ((orig-est) / orig) * 100

##### Mean of past_avg_amount_annuity of infringements

In [123]:
dp_count = add_laplace_noise(original_count, sensitivity_num_infringed_past_avg_amt_an, 0.01)
print(original_count, dp_count, percentage_error(original_count, dp_count))

TypeError: unsupported operand type(s) for /: 'function' and 'float'

##### Mean of people age of infringements

In [129]:
dp_count = add_laplace_noise(original_count, sensitivity_age, 0.01)
print(original_count, dp_count, percentage_error(original_count, dp_count))

24825 24824.946790178597 0.0002143396632530148
